In [1]:
!pip uninstall -y scikit-learn

Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2


In [2]:
!pip install scikit-learn==1.3.2 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 104.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nilearn 0.11.1 requires scikit-learn>=1.4.0, but you have scikit-learn 1.3.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [3]:
!pip install -U imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 104.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.13.0
    Uninstalling imbalanced-learn-0.13.0:
      Successfully uninstalled imbalanced-learn-0.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 wh

In [4]:
import os
import logging

# Suppress TensorFlow and related logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Hide TensorFlow INFO/WARNING logs
logging.getLogger('tensorflow').setLevel(logging.ERROR)

# Optional: Hide absl and factory register warnings
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Ensures single GPU is used

# Suppress other system-wide logs like XLA and absl
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logs
os.environ['TF_ABS_WARNINGS'] = '0'  # Suppress absl logs
os.environ['XLA_FLAGS'] = '--xla_disable_mlir=1'  # Disable XLA info logs

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.applications.convnext import preprocess_input as convnext_preprocess
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Import SMOTE for handling imbalanced data
from imblearn.over_sampling import SMOTE  # Import SMOTE


E0000 00:00:1758891445.152299      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758891445.208061      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# =================== PATHS ====================
train_val_path = '/kaggle/input/kaggle-skin-desease/skin-disease-datasaet/train_set'
test_path = '/kaggle/input/kaggle-skin-desease/skin-disease-datasaet/test_set'


In [6]:
# Data Generator
datagen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
    shear_range=0.1, zoom_range=0.1, horizontal_flip=True,
    brightness_range=[0.8, 1.2], fill_mode='reflect'
)


In [7]:
# =================== DATA LOADING ====================
def load_data(path):
    data = []
    for folder in sorted(os.listdir(path)):
        folder_path = os.path.join(path, folder)
        if not os.path.isdir(folder_path):
            continue
        files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if len(files) < 2:
            continue
        num_train = int(0.8 * len(files))
        files_train = random.sample(files, num_train)
        files_val = list(set(files) - set(files_train))
        for file in files_train + files_val:
            file_path = os.path.join(folder_path, file)
            img = cv2.imread(file_path)
            if img is not None:
                img = cv2.resize(img, (224, 224))
                data.append((img, folder))
    return data

In [8]:
# Load training and testing data
train_val_data = load_data(train_val_path)
test_data = load_data(test_path)

X_train, y_train = zip(*train_val_data)
X_test, y_test = zip(*test_data)

In [9]:
# Label encoding
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)
num_classes = len(le.classes_)

In [10]:
# One-hot encoding
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

In [11]:
# Apply SMOTE
def apply_smote(X, y):
    X_np = np.array(X)
    X_flat = X_np.reshape(len(X_np), -1)
    smote = SMOTE()  # Create SMOTE object
    X_res, y_res = smote.fit_resample(X_flat, y)
    return X_res.reshape(-1, 224, 224, 3), y_res

X_train_smote, y_train_smote = apply_smote(X_train, y_train_encoded)

In [12]:
# =================== TRAINING ConvNeXt ====================
print("\nTraining ConvNeXt...")
X_train_convnext = convnext_preprocess(np.array(X_train_smote))
X_test_convnext = convnext_preprocess(np.array(X_test))

convnext_base = ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
convnext_base.trainable = False

x = GlobalMaxPooling2D()(convnext_base.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

convnext_model = Model(inputs=convnext_base.input, outputs=output)
convnext_model.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])

convnext_history = convnext_model.fit(
    X_train_convnext, to_categorical(y_train_smote, num_classes),
    validation_data=(X_test_convnext, y_test_one_hot),
    epochs=10, batch_size=32,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
    verbose=1
)



Training ConvNeXt...


I0000 00:00:1758891463.963864      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
# Save the trained model after training
convnext_model.save('skin_disease_model.h5')  # Save model as .h5 file
print("Model saved as skin_disease_model.h5")

In [ ]:
# =================== EVALUATION ====================
# Accuracy Plot
plt.figure(figsize=(10,6))
plt.plot(convnext_history.history['accuracy'], label='Train Accuracy')
plt.plot(convnext_history.history['val_accuracy'], label='Val Accuracy')
plt.title("ConvNeXt Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# Loss Plot
plt.figure(figsize=(10,6))
plt.plot(convnext_history.history['loss'], label='Train Loss')
plt.plot(convnext_history.history['val_loss'], label='Val Loss')
plt.title("ConvNeXt Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Predictions
convnext_preds = np.argmax(convnext_model.predict(X_test_convnext), axis=1)

# Confusion Matrix
plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix(y_test_encoded, convnext_preds), annot=True, fmt='d', cmap='Blues',
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title("ConvNeXt Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
# Classification Report
print("\nConvNeXt Classification Report:")
print(classification_report(y_test_encoded, convnext_preds, target_names=le.classes_))

# ROC Curve
plt.figure(figsize=(10, 8))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(y_test_one_hot[:, i], convnext_model.predict(X_test_convnext)[:, i])
    plt.plot(fpr, tpr, label=f'{le.classes_[i]} (AUC={auc(fpr,tpr):.2f})')

plt.plot([0,1],[0,1],'k--')
plt.title("ConvNeXt ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()

print("✅ ConvNeXt training and evaluation completed!")